In [2]:
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import getdist
from getdist import plots, MCSamples
import pydelfi.priors as priors
import pydelfi.ndes as ndes
import pydelfi.delfi as delfi
import pydelfi.score as score
import simulators.cosmic_shear.cosmic_shear as cosmic_shear
import pickle
import tensorflow_probability as tfp
tfd = tfp.distributions

## Set up the simulator
This must have the signature `simulator(parameters, seed, args, batch)` -> `np.array([batch, ndata])`

In [3]:
CosmicShearSimulator = cosmic_shear.TomographicCosmicShear(pz = pickle.load(open('simulators/cosmic_shear/pz_5bin.pkl', 'rb')),
                                                           lmin = 10, lmax = 1000, n_ell_bins = 5, 
                                                           sigma_e = 0.3, nbar = 30, Area = 15000)

def simulator(theta, seed, simulator_args, batch=1):
    return CosmicShearSimulator.simulate(theta, seed)

simulator_args = None

## Set up the prior

In [4]:
lower = np.array([0, 0.4, 0, 0.4, 0.7]).astype('float32')
upper = np.array([1, 1.2, 0.1, 1.0, 1.3]).astype('float32')
prior_mean = np.array([0.3, 0.8, 0.05, 0.70, 0.96]).astype('float32')
prior_covariance = (np.eye(5)*np.array([0.1, 0.1, 0.05, 0.3, 0.3])**2).astype('float32')
prior_stddev = np.sqrt(np.diag(prior_covariance))

prior = tfd.Blockwise([tfd.TruncatedNormal(loc=prior_mean[i], scale=prior_stddev[i], low=lower[i], high=upper[i]) for i in range(5)])

## Set up the compressor
Must have the signature `compressor(data, args)` -> `np.array([n_summaries])`<br>
In this case we are going to do Wishart score compression.

In [5]:
# Fiducial parameters
theta_fiducial = np.array([0.3, 0.8, 0.05, 0.70, 0.96])

# Expected support of Wishart likelihood (fiducial inverse power spectrum)
C = CosmicShearSimulator.power_spectrum(theta_fiducial)
Cinv = np.array([np.linalg.inv(C[l,:,:]) for l in range(CosmicShearSimulator.n_ell_bins)])

# Degrees of freedom (effective number of modes per band power)
nl = CosmicShearSimulator.nl

# Calculate derivatives of the expected power spectrum
step = np.array(abs(theta_fiducial)*np.array([0.05, 0.05, 0.05, 0.05, 0.05]))
dCdt = CosmicShearSimulator.compute_derivatives(theta_fiducial, step)

# Define compression as score-MLE of a Wishart likelihood
Compressor = score.Wishart(theta_fiducial, nl, Cinv, dCdt, prior_mean=prior_mean, prior_covariance=prior_covariance)

# Pull out Fisher matrix inverse
Finv = Compressor.Finv

# Compressor function: This must have the form compressor(data, args) -> compressed summaries (pseudoMLE)
def compressor(d, compressor_args):
    return Compressor.scoreMLE(d)
compressor_args = None

## Generate mock data vector

In [5]:
seed = 0
data = simulator(theta_fiducial, seed, simulator_args)
compressed_data = compressor(data, compressor_args)

## Create ensemble of NDEs

In [6]:
NDEs = [ndes.ConditionalMaskedAutoregressiveFlow(
            n_parameters=5,
            n_data=5,
            n_mades=5,
            n_hidden=[30,30], 
            activation=tf.keras.layers.LeakyReLU(0.01),
            kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-5, seed=None),
            all_layers=True)]

NDEs += [ndes.MixtureDensityNetwork(
            n_parameters=5,
            n_data=5, 
            n_components=i+1,
            n_hidden=[30], 
            activation=tf.keras.layers.LeakyReLU(0.01))
        for i in range(5)]

NDEs += [ndes.SinhArcSinhMADE(
            n_parameters=5,
            n_data=5,
            n_hidden=[64],
            activation=tf.tanh,
            kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-5, seed=None),
            bias_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=1e-5, seed=None)
        )]

Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
`batch_shape` and `event_shape` args are deprecated. Please use `tfd.Sample`, `tfd.Independent`, and broadcasted parameters of the base distribution instead. For example, replace `tfd.TransformedDistribution(tfd.Normal(0., 1.), tfb.Exp(), batch_shape=[2, 3], event_shape=[4])` with `tfd.TransformedDistrbution(tfd.Sample(tfd.Normal(tf.zeros([2, 3]), 1.),sample_shape=[4]), tfb.Exp())` or `tfd.TransformedDistribution(tfd.Independent(tfd.Normal(tf.zeros([2, 3, 4]), 1.), reinterpreted_batch_ndims=1), tfb.Exp())`.
Instructions for updating:
`batch_shape` and `event_shape` args are deprecated. Please use `tfd.Sample`, `tfd.Independent`, and broadcasted parameters of the base distribution instead. For example, replace `tfd.TransformedDistribution(tfd.Normal(0., 1.), tfb.Exp(), batch_shape=[2, 3], event_shape=[4])` with `tfd.TransformedDistrbution(tfd.Sample(tfd.Normal(tf.zeros([2, 3

## Create DELFI object

In [ ]:
DelfiEnsemble = delfi.Delfi(compressed_data, prior, NDEs, 
                            Finv=Finv, 
                            theta_fiducial=theta_fiducial,
                            param_limits = [lower, upper],
                            param_names=['\Omega_m', 'S_8', '\Omega_b', 'h', 'n_s'], 
                            results_dir="simulators/cosmic_shear/results",
                            filename="cosmic_shear",
                            optimiser=tf.keras.optimizers.Adam(lr=1e-4),
                            optimiser_arguments=None,
                            dtype=tf.float32,
                            posterior_chain_length=200,
                            nwalkers=500,
                            input_normalization="fisher")

## Fisher pre-training to initialize NDEs

In [1]:
# DelfiEnsemble.fisher_pretraining(n_batch=5000, epochs=1000, patience=20, plot=True)

## Sequential Neural Likelihood

In [ ]:
# Initial samples, batch size for population samples, number of populations
n_initial = 200
n_batch = 200
n_populations = 39

# Do the SNL training
DelfiEnsemble.sequential_training(simulator, compressor, n_initial, n_batch, n_populations, patience=10, plot=True, save_intermediate_posteriors=True)